In [19]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
import time

In [20]:
df = pd.read_csv('iris.csv')
X = df.iloc[:, :-1].values
Y = df.iloc[:, -1].values
idx = np.arange(len(X))
np.random.shuffle(idx)
split = int(0.8 * len(X))
tr = X[idx[:split]]
ts = X[idx[split:]]
mu = np.mean(tr, axis=0)
std = np.std(tr, axis=0)
tr = (tr - mu) / std
ts = (ts - mu) / std
trl = Y[idx[:split]]
tsl = Y[idx[split:]]

In [21]:
def bruteknn(tr, tl, ts, k, m='euclidean', p=3):
    def dist(a, b):
        if m == 'euclidean':
            return np.sqrt(np.sum((a - b) ** 2))
        elif m == 'manhattan':
            return np.sum(np.abs(a - b))
        elif m == 'minkowski':
            return np.sum(np.abs(a - b) ** p) ** (1 / p)

    res = []
    for x in ts:
        d = [ (dist(x, tr[i]), tl[i]) for i in range(len(tr)) ]
        d.sort()
        lbls = [lbl for _, lbl in d[:k]]
        res.append(max(set(lbls), key=lbls.count))
    return res

In [22]:
def kdknn(tr, tl, ts, k, m='euclidean', p=3):
    def dist(a, b):
        if m == 'euclidean':
            return np.sqrt(np.sum((a - b) ** 2))
        elif m == 'manhattan':
            return np.sum(np.abs(a - b))
        elif m == 'minkowski':
            return np.sum(np.abs(a - b) ** p) ** (1 / p)

    res = []
    for x in ts:
        d = np.array([dist(x, t) for t in tr])
        idx = np.argsort(d)[:k]
        lbls = tl[idx]
        val = max(set(lbls), key=list(lbls).count)
        res.append(val)
    return res


In [31]:
def get_cm(y_true, y_pred, labels):
    n = len(labels)
    cm = np.zeros((n, n), dtype=int)
    for a, b in zip(y_true, y_pred):
        i = labels.index(a)
        j = labels.index(b)
        cm[i][j] += 1
    return cm

def get_metrics(cm):
    TP = np.diag(cm)
    FP = np.sum(cm, axis=0) - TP
    FN = np.sum(cm, axis=1) - TP
    prec = np.mean(np.where((TP + FP) != 0, TP / (TP + FP), 0))
    rec = np.mean(np.where((TP + FN) != 0, TP / (TP + FN), 0))
    f1 = np.mean(np.where((prec + rec) != 0, 2 * prec * rec / (prec + rec), 0))
    acc = np.sum(TP) / np.sum(cm)
    return prec, rec, f1, acc

In [40]:
k_values = [1, 3, 5, 7]
metrics = ['euclidean', 'manhattan', 'minkowski']
labels = list(np.unique(Y))
results = []

for k in k_values:
    for m in metrics:
        pred1 = bruteknn(tr, trl, ts, k=k, m=m)
        pred2 = kdknn(tr, trl, ts, k=k, m=m)
        cm1 = get_cm(tsl, pred1, labels)
        cm2 = get_cm(tsl, pred2, labels)
        p1, r1, f1s, a1 = get_metrics(cm1)
        p2, r2, f2s, a2 = get_metrics(cm2)
        results.append((k, m, 'BruteForce', cm1, p1, r1, f1s, a1))
        results.append((k, m, 'KDTree', cm2, p2, r2, f2s, a2))

out = ""
for r in results:
    k, m, method, cm, p, r_, f1s, a = r
    out += f"\nK = {k}, Metric = {m.upper()}, Method = {method}\n"
    out += f"Confusion Matrix:\n{cm}\n"
    out += f"Precision: {p:.2f}, Recall: {r_:.2f}, F1: {f1s:.2f}, Accuracy: {a:.2f}\n"

print(out)


K = 1, Metric = EUCLIDEAN, Method = BruteForce
Confusion Matrix:
[[ 9  0  0]
 [ 0 10  0]
 [ 0  0 11]]
Precision: 1.00, Recall: 1.00, F1: 1.00, Accuracy: 1.00

K = 1, Metric = EUCLIDEAN, Method = KDTree
Confusion Matrix:
[[ 9  0  0]
 [ 0 10  0]
 [ 0  0 11]]
Precision: 1.00, Recall: 1.00, F1: 1.00, Accuracy: 1.00

K = 1, Metric = MANHATTAN, Method = BruteForce
Confusion Matrix:
[[ 9  0  0]
 [ 0 10  0]
 [ 0  0 11]]
Precision: 1.00, Recall: 1.00, F1: 1.00, Accuracy: 1.00

K = 1, Metric = MANHATTAN, Method = KDTree
Confusion Matrix:
[[ 9  0  0]
 [ 0 10  0]
 [ 0  0 11]]
Precision: 1.00, Recall: 1.00, F1: 1.00, Accuracy: 1.00

K = 1, Metric = MINKOWSKI, Method = BruteForce
Confusion Matrix:
[[ 9  0  0]
 [ 0 10  0]
 [ 0  0 11]]
Precision: 1.00, Recall: 1.00, F1: 1.00, Accuracy: 1.00

K = 1, Metric = MINKOWSKI, Method = KDTree
Confusion Matrix:
[[ 9  0  0]
 [ 0 10  0]
 [ 0  0 11]]
Precision: 1.00, Recall: 1.00, F1: 1.00, Accuracy: 1.00

K = 3, Metric = EUCLIDEAN, Method = BruteForce
Confusion 